## PRODUCT TYPE CLASSIFICATION USING TEXT BISON LLM FOR AEROSOL

### Import Packages

In [1]:
import pandas as pd
import json
import os
import mapply
from google.cloud import bigquery
import gcsfs
import ast
import time
from tqdm import tqdm
client = bigquery.Client()
from vertexai.preview.language_models import TextGenerationModel
from smart_open import smart_open

mapply.init(
    n_workers=-1,
    chunk_size=10000,
    max_chunks_per_worker=0,
    progressbar=True
)
tqdm.pandas(desc='progress bar')
pd.set_option('display.max_columns', None)


2023-12-13 10:30:28.123898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 10:30:29.407824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-12-13 10:30:29.408016: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

### Tables used: 
<ol>
    <li> `wmt-dca-catalog-dq-dev.POD2.POD2_CORE` - CORE TABLE SEE TABLE DIRECTORY </li>
</ol>

[TABLE DIRECTORY LINK](https://teams.wal-mart.com/:x:/r/sites/E2EFulfillmentDataFoundation/Shared%20Documents/Accurate%20Data/Deloitte%20Item%20Accuracy%20Project/02.%20Score%20and%20Clean/06.%20Final%20Deliverables/Technical%20Guide%20Supporting%20Files/score_and_clean_table_directory.xlsx?d=w67588517d0d54c4982d2242973906a28&csf=1&web=1&e=9tsU3Z)

#### Query to get ALL Product types and product names

In [2]:
total  = """
SELECT WPID,PROD_TYPE_NM,product_name as PROD_NM FROM `wmt-dca-catalog-dq-dev.POD2.POD2_CORE`
"""
total = client.query(total ).to_dataframe()

#### Taking Sample 3 random products from each product type group.

In [3]:
total_sample = total.groupby('PROD_TYPE_NM').sample(n=3,random_state=42,replace=True).reset_index(drop=True)

In [4]:
total_sample_pivot = total_sample[['PROD_TYPE_NM','PROD_NM']]

In [5]:
# creating samples column
total_sample_pivot['samples'] = [1,2,3]*(int(len(total_sample_pivot)/3))

In [6]:
# creating pivot table
total_sample_pivot = total_sample_pivot.pivot(index='PROD_TYPE_NM',columns='samples',values='PROD_NM').reset_index()

In [7]:
#combing all three products into single list and assiging into example products column
total_sample_pivot['example_products'] = total_sample_pivot.mapply(lambda x: [x[1],x[2],x[3]],axis=1)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
#creating a dictionary with product types and list of products
product_types = {i:a for i,a in zip(list(total_sample_pivot['PROD_TYPE_NM']),list(total_sample_pivot['example_products']))}

In [9]:
total_sample_pivot

samples,PROD_TYPE_NM,1,2,3,example_products
0,"3-in-1 Shampoo, Conditioner & Body Washes",Suave Kids Naturals 3-in-1 Shampoo Conditioner...,Dove Love Your Chic edge 3-in-1 Short & Sweet ...,American Crew Classic 3-in-1 Shampoo plus Cond...,[Suave Kids Naturals 3-in-1 Shampoo Conditione...
1,360° Cameras,Ricoh TH-2 Weatherproof Hard Case for Theta M1...,Insta360 Invisible - Selfie stick - for Insta3...,Nikon KeyMission 360 4K Action Camera,[Ricoh TH-2 Weatherproof Hard Case for Theta M...
2,3D Glasses,3X Red-Blue / Cyan Anaglyph Simple Style 3D Gl...,Future Sound of Jazz 9 [Audio CD] Various Arti...,Cinema Adult 3D Glasses 3D Stereoscopic 3D Pol...,[3X Red-Blue / Cyan Anaglyph Simple Style 3D G...
3,3D Printer Filaments,"GEEETECH Green PETG Filament 1.75mm, Upgrade S...",3D-Fuel WorkDay PLA Snow White 1.75mm 1Kg 3D F...,"MakerBot 1.75mm PLA Filament for Sketch, White...","[GEEETECH Green PETG Filament 1.75mm, Upgrade ..."
4,3D Printer Parts,LYUMO DRV8825/A4988 42 Stepper Motor Driver Mo...,Impresa 2-Pack 3D Printer Glass Print Bed for ...,"Magnetic Build , Matte 3D Printer Bed Cover Fo...",[LYUMO DRV8825/A4988 42 Stepper Motor Driver M...
...,...,...,...,...,...
6638,Zithers,Children Practice Guzheng Chinese String Instr...,Children Practice Guzheng Chinese String Instr...,Hemoton Children Practice Guzheng Chinese Stri...,[Children Practice Guzheng Chinese String Inst...
6639,default,"CREATE YOUR OWN Foam Board, horizontal, Wall D...",Clutch Friction Plates And Gasket Kit for Hond...,"2PK National Hardware N270-316 Kickplate, Sati...","[CREATE YOUR OWN Foam Board, horizontal, Wall ..."
6640,pH Meters,Digital PH Meter + LCD TDS EC Water Purity PPM...,Karlge Salinity Meter Digital PenType Salinity...,Docooler PH Meter Calibration Point 3 Type PH ...,[Digital PH Meter + LCD TDS EC Water Purity PP...
6641,pH Test Strips,OTVIAP 80pcs PH Test Strips Alkaline Paper Uri...,Olympian Labs pH Papers - 6.0-8.0 Range - 15 ft,160 Strips 1-14 Test Paper PH Indicator Alkali...,[OTVIAP 80pcs PH Test Strips Alkaline Paper Ur...


#### generating prompts for all product types

In [10]:


total_sample_pivot['prompt'] = total_sample_pivot.mapply(lambda x: f"""You are an expert at classifying products for risk. Tell me the risk level that the product category contains aerosol spray items.

product category: Books
example products:
Dear Peter Rabbit (Hardcover)
The Wages of Sin (Paperback)
resonse: low

product category: Hair Sprays
example product:
Paul Mitchell Firm Style Freeze and Shine Super Hairspray, 16.9 oz (Pack of 2)
Goldwell Kerasilk Style Texturizing Finish Spray - 5.6 oz
response: high

product category: Automotive Touch-Up Paints
example products:
ERA Paints (1D6/U1D6 Silver Sky Metallic) Compatible with Toyota Tundra 2001-2020
SUNBURST GOLD M
response: medium

product category: Acne Treatment Systems
example products:
Acne Treatment, Retinoid Treatment for Face with 0.1% Adapalene
Epionce Purifying Spot Gel 0.34 Ounce
response: medium

now its your turn

product category: {x['PROD_TYPE_NM']}
example products:
{x[1]}
{x[2]}
{x[3]}

response:""",axis=1)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/1 [00:00<?, ?it/s]

#### function to perform  text bison LLM online predictions

In [11]:

def get_ptype_aero_status(prompt):
    text_model = TextGenerationModel.from_pretrained("text-bison@001")
    response = text_model.predict(prompt=prompt, temperature = 0.0, top_k = 1).text
    time.sleep(1)
    return response

In [12]:
# samples to test results
aerosol_sample = total_sample_pivot.sample(n=10).reset_index(drop=True)

In [13]:
aerosol_sample['prompt'][0]

'You are an expert at classifying products for risk. Tell me the risk level that the product category contains aerosol spray items.\n\nproduct category: Books\nexample products:\nDear Peter Rabbit (Hardcover)\nThe Wages of Sin (Paperback)\nresonse: low\n\nproduct category: Hair Sprays\nexample product:\nPaul Mitchell Firm Style Freeze and Shine Super Hairspray, 16.9 oz (Pack of 2)\nGoldwell Kerasilk Style Texturizing Finish Spray - 5.6 oz\nresponse: high\n\nproduct category: Automotive Touch-Up Paints\nexample products:\nERA Paints (1D6/U1D6 Silver Sky Metallic) Compatible with Toyota Tundra 2001-2020\nSUNBURST GOLD M\nresponse: medium\n\nproduct category: Acne Treatment Systems\nexample products:\nAcne Treatment, Retinoid Treatment for Face with 0.1% Adapalene\nEpionce Purifying Spot Gel 0.34 Ounce\nresponse: medium\n\nnow its your turn\n\nproduct category: TV & Monitor Mounts\nexample products:\nTV Wall Mount Full Motion Swivel Articulating Bracket Most 32-55 LED LCD OLED Plasma Flat

In [14]:

%%time
aerosol_sample['classification'] = aerosol_sample['prompt'].progress_apply(lambda x: get_ptype_aero_status(x))


progress bar: 100%|██████████| 10/10 [00:14<00:00,  1.40s/it]

CPU times: user 346 ms, sys: 63.5 ms, total: 410 ms
Wall time: 14 s


#### creating jsonl file with prompts for batch prediction

In [15]:
with open('aero_prompts.jsonl', 'w') as fp:
    for item in total_sample_pivot['prompt'].to_list():
        data = {"prompt": item}
        fp.write(json.dumps(data) + "\n")

In [73]:
#copying jsonl file into gcs bucket
!gsutil cp aero_prompts.jsonl gs://phase2_scoreclean/pod2/aerosol_ptype_classification

Copying file://aero_prompts.jsonl [Content-Type=application/octet-stream]...
/ [1 files][  7.6 MiB/  7.6 MiB]                                                
Operation completed over 1 objects/7.6 MiB.                                      


#### Performing batch predictions

In [74]:
from vertexai.preview.language_models import TextGenerationModel
text_model = TextGenerationModel.from_pretrained("text-bison")
batch_prediction_job = text_model.batch_predict(
  source_uri=["gs://phase2_scoreclean/pod2/aerosol_ptype_classification/aero_prompts.jsonl"],
  destination_uri_prefix="gs://phase2_scoreclean/pod2/aerosol_ptype_classification/results/",
  # Optional:
  model_parameters={
      "maxOutputTokens": "200",
      "temperature": "0.0",
      "topK": "1",
  },
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/3795033446342983680?project=613500287760


/opt/conda/lib/python3.10/site-packages/vertexai/language_models/_language_models.py:924: UserWarning: source_uri is deprecated, use dataset instead.
  warnings.warn("source_uri is deprecated, use dataset instead.")


BatchPredictionJob projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/613500287760/locations/us-central1/batchPredictionJobs/3795033446342983680 current state:
JobState.JOB

#### reading batch prediction results and converting into csv file

In [75]:
df = pd.read_json('gs://phase2_scoreclean/pod2/aerosol_ptype_classification/results/prediction-model-2023-10-25T20:00:26.707816Z/000000000000.jsonl',lines=True)

In [86]:
df['product_type'] = df['instance'].apply(lambda x: x['prompt'].split('product category:')[-1].split('\n')[0].strip())

In [87]:
df['classification'] = df['predictions'].apply(lambda x: x[0]['content'].strip() if pd.isnull(x)!=True else None)

#### Define a file names and save it in a csv file in the GCS bucket

In [89]:
df.to_csv('gs://phase2_scoreclean/pod2/aerosol/aerosol_product_type_classification.csv',index=False)

In [12]:
# library version
# Package                                  Version
# ---------------------------------------- ---------------
# absl-py                                  1.4.0
# aiohttp                                  3.8.5
# aiohttp-cors                             0.7.0
# aiorwlock                                1.3.0
# aiosignal                                1.3.1
# ansiwrap                                 0.8.4
# anyio                                    3.7.1
# apache-beam                              2.46.0
# argon2-cffi                              23.1.0
# argon2-cffi-bindings                     21.2.0
# array-record                             0.4.1
# arrow                                    1.2.3
# asttokens                                2.4.0
# astunparse                               1.6.3
# async-lru                                2.0.4
# async-timeout                            4.0.3
# attrs                                    23.1.0
# Babel                                    2.12.1
# backcall                                 0.2.0
# backoff                                  2.2.1
# backports.functools-lru-cache            1.6.5
# beatrix-jupyterlab                       2023.814.150030
# beautifulsoup4                           4.12.2
# bleach                                   6.0.0
# blessed                                  1.20.0
# boltons                                  23.0.0
# Brotli                                   1.1.0
# cached-property                          1.5.2
# cachetools                               4.2.4
# certifi                                  2023.7.22
# cffi                                     1.15.1
# charset-normalizer                       3.2.0
# click                                    8.1.7
# cloud-tpu-client                         0.10
# cloud-tpu-profiler                       2.4.0
# cloudpickle                              2.2.1
# colorama                                 0.4.6
# colorful                                 0.5.5
# comm                                     0.1.4
# conda                                    23.7.4
# conda-package-handling                   2.2.0
# conda_package_streaming                  0.9.0
# contourpy                                1.1.1
# crcmod                                   1.7
# cryptography                             41.0.4
# cycler                                   0.11.0
# Cython                                   3.0.2
# dacite                                   1.8.1
# db-dtypes                                1.1.1
# debugpy                                  1.8.0
# decorator                                5.1.1
# defusedxml                               0.7.1
# Deprecated                               1.2.14
# dill                                     0.3.7
# distlib                                  0.3.7
# dm-tree                                  0.1.8
# docker                                   6.1.3
# docopt                                   0.6.2
# docstring-parser                         0.15
# entrypoints                              0.4
# etils                                    1.5.0
# exceptiongroup                           1.1.3
# executing                                1.2.0
# explainable-ai-sdk                       1.3.3
# Farama-Notifications                     0.0.4
# fastapi                                  0.103.1
# fastavro                                 1.8.3
# fasteners                                0.19
# fastjsonschema                           2.18.0
# filelock                                 3.12.4
# flatbuffers                              23.5.26
# fonttools                                4.42.1
# fqdn                                     1.5.1
# frozenlist                               1.4.0
# fsspec                                   2023.9.2
# gast                                     0.4.0
# gcsfs                                    2023.9.2
# gitdb                                    4.0.10
# GitPython                                3.1.37
# google-api-core                          2.11.1
# google-api-python-client                 1.8.0
# google-apitools                          0.5.31
# google-auth                              2.23.0
# google-auth-httplib2                     0.1.1
# google-auth-oauthlib                     1.1.0
# google-cloud-aiplatform                  1.33.1
# google-cloud-artifact-registry           1.8.3
# google-cloud-bigquery                    3.11.4
# google-cloud-bigquery-storage            2.16.2
# google-cloud-bigtable                    1.7.3
# google-cloud-core                        2.3.3
# google-cloud-datastore                   1.15.5
# google-cloud-dlp                         3.12.3
# google-cloud-language                    1.3.2
# google-cloud-monitoring                  2.15.1
# google-cloud-pubsub                      2.18.4
# google-cloud-pubsublite                  1.8.3
# google-cloud-recommendations-ai          0.7.1
# google-cloud-resource-manager            1.10.4
# google-cloud-spanner                     3.40.1
# google-cloud-storage                     2.11.0
# google-cloud-videointelligence           1.16.3
# google-cloud-vision                      3.4.4
# google-crc32c                            1.5.0
# google-pasta                             0.2.0
# google-resumable-media                   2.6.0
# googleapis-common-protos                 1.60.0
# gpustat                                  1.0.0
# greenlet                                 2.0.2
# grpc-google-iam-v1                       0.12.6
# grpcio                                   1.48.0
# grpcio-status                            1.48.0
# gviz-api                                 1.10.0
# gymnasium                                0.28.1
# h11                                      0.14.0
# h5py                                     3.9.0
# hdfs                                     2.7.2
# htmlmin                                  0.1.12
# httplib2                                 0.21.0
# idna                                     3.4
# ImageHash                                4.3.1
# imageio                                  2.31.4
# importlib-metadata                       6.8.0
# importlib-resources                      6.1.0
# ipykernel                                6.25.2
# ipython                                  8.15.0
# ipython-genutils                         0.2.0
# ipython-sql                              0.5.0
# ipywidgets                               8.1.1
# isoduration                              20.11.0
# jaraco.classes                           3.3.0
# jax-jumpy                                1.0.0
# jedi                                     0.19.0
# jeepney                                  0.8.0
# Jinja2                                   3.1.2
# joblib                                   1.3.2
# json5                                    0.9.14
# jsonpatch                                1.33
# jsonpointer                              2.4
# jsonschema                               4.19.1
# jsonschema-specifications                2023.7.1
# jupyter_client                           7.4.9
# jupyter_core                             5.3.1
# jupyter-events                           0.7.0
# jupyter-http-over-ws                     0.0.8
# jupyter-lsp                              2.2.0
# jupyter-server                           1.24.0
# jupyter-server-mathjax                   0.2.6
# jupyter_server_proxy                     4.1.0
# jupyter_server_terminals                 0.4.4
# jupyterlab                               3.4.8
# jupyterlab_git                           0.43.0
# jupyterlab-pygments                      0.2.2
# jupyterlab_server                        2.25.0
# jupyterlab-widgets                       3.0.9
# jupytext                                 1.15.2
# keras                                    2.11.0
# keras-core                               0.1.7
# keras-tuner                              1.4.1
# keyring                                  24.2.0
# keyrings.google-artifactregistry-auth    1.1.2
# kfp                                      2.3.0
# kfp-pipeline-spec                        0.2.2
# kfp-server-api                           2.0.1
# kiwisolver                               1.4.5
# kt-legacy                                1.0.5
# kubernetes                               26.1.0
# lazy_loader                              0.3
# libclang                                 16.0.6
# llvmlite                                 0.41.0
# lz4                                      4.3.2
# mapply                                   0.1.21
# Markdown                                 3.4.4
# markdown-it-py                           3.0.0
# MarkupSafe                               2.0.1
# matplotlib                               3.8.0
# matplotlib-inline                        0.1.6
# mdit-py-plugins                          0.4.0
# mdurl                                    0.1.2
# mistune                                  3.0.1
# more-itertools                           10.1.0
# msgpack                                  1.0.6
# multidict                                6.0.4
# multimethod                              1.10
# multiprocess                             0.70.15
# namex                                    0.0.7
# nb-conda                                 2.2.1
# nb-conda-kernels                         2.3.1
# nbclassic                                1.0.0
# nbclient                                 0.8.0
# nbconvert                                7.8.0
# nbdime                                   3.2.0
# nbformat                                 5.9.2
# nest-asyncio                             1.5.6
# networkx                                 3.1
# notebook                                 6.5.6
# notebook-executor                        0.2
# notebook_shim                            0.2.3
# numba                                    0.58.0
# numpy                                    1.23.5
# nvidia-ml-py                             11.495.46
# oauth2client                             4.1.3
# oauthlib                                 3.2.2
# objsize                                  0.6.1
# opencensus                               0.11.3
# opencensus-context                       0.1.3
# opentelemetry-api                        1.20.0
# opentelemetry-exporter-otlp              1.20.0
# opentelemetry-exporter-otlp-proto-common 1.20.0
# opentelemetry-exporter-otlp-proto-grpc   1.20.0
# opentelemetry-exporter-otlp-proto-http   1.20.0
# opentelemetry-proto                      1.20.0
# opentelemetry-sdk                        1.20.0
# opentelemetry-semantic-conventions       0.41b0
# opt-einsum                               3.3.0
# orjson                                   3.9.7
# overrides                                6.5.0
# packaging                                23.1
# pandas                                   2.0.3
# pandas-gbq                               0.19.2
# pandas-profiling                         3.6.6
# pandocfilters                            1.5.0
# papermill                                2.4.0
# parso                                    0.8.3
# pathos                                   0.3.1
# patsy                                    0.5.3
# pexpect                                  4.8.0
# phik                                     0.12.3
# pickleshare                              0.7.5
# Pillow                                   10.0.1
# pip                                      23.2.1
# pkgutil_resolve_name                     1.3.10
# platformdirs                             3.10.0
# plotly                                   5.17.0
# pluggy                                   1.3.0
# pox                                      0.3.3
# ppft                                     1.7.6.7
# prettytable                              3.9.0
# prometheus-client                        0.17.1
# promise                                  2.3
# prompt-toolkit                           3.0.39
# proto-plus                               1.22.3
# protobuf                                 3.19.6
# psutil                                   5.9.3
# ptyprocess                               0.7.0
# pure-eval                                0.2.2
# py-spy                                   0.3.14
# pyarrow                                  9.0.0
# pyasn1                                   0.5.0
# pyasn1-modules                           0.3.0
# pycosat                                  0.6.4
# pycparser                                2.21
# pydantic                                 1.10.12
# pydata-google-auth                       1.8.2
# pydot                                    1.4.2
# Pygments                                 2.16.1
# PyJWT                                    2.8.0
# pymongo                                  3.13.0
# pyOpenSSL                                23.2.0
# pyparsing                                3.1.1
# PySocks                                  1.7.1
# python-dateutil                          2.8.2
# python-json-logger                       2.0.7
# pytz                                     2023.3.post1
# pyu2f                                    0.1.5
# PyWavelets                               1.4.1
# PyYAML                                   6.0.1
# pyzmq                                    24.0.1
# ray                                      2.7.0
# ray-cpp                                  2.7.0
# referencing                              0.30.2
# regex                                    2023.8.8
# requests                                 2.31.0
# requests-oauthlib                        1.3.1
# requests-toolbelt                        0.10.1
# retrying                                 1.3.3
# rfc3339-validator                        0.1.4
# rfc3986-validator                        0.1.1
# rich                                     13.5.3
# rpds-py                                  0.10.3
# rsa                                      4.9
# ruamel.yaml                              0.17.32
# ruamel.yaml.clib                         0.2.7
# scikit-image                             0.21.0
# scikit-learn                             1.3.1
# scipy                                    1.11.2
# seaborn                                  0.12.2
# SecretStorage                            3.3.3
# Send2Trash                               1.8.2
# setuptools                               68.2.2
# Shapely                                  1.8.5.post1
# simpervisor                              1.0.0
# six                                      1.16.0
# smart-open                               6.4.0
# smmap                                    5.0.1
# sniffio                                  1.3.0
# soupsieve                                2.5
# SQLAlchemy                               2.0.21
# sqlparse                                 0.4.4
# stack-data                               0.6.2
# starlette                                0.27.0
# statsmodels                              0.14.0
# tabulate                                 0.9.0
# tangled-up-in-unicode                    0.2.0
# tenacity                                 8.2.3
# tensorboard                              2.11.2
# tensorboard-data-server                  0.6.1
# tensorboard-plugin-profile               2.13.1
# tensorboard-plugin-wit                   1.8.1
# tensorboardX                             2.6
# tensorflow                               2.11.0
# tensorflow-cloud                         0.1.16
# tensorflow-datasets                      4.9.0
# tensorflow-estimator                     2.11.0
# tensorflow-hub                           0.14.0
# tensorflow-io                            0.29.0
# tensorflow-io-gcs-filesystem             0.29.0
# tensorflow-metadata                      0.14.0
# tensorflow-probability                   0.21.0
# tensorflow-serving-api                   2.11.0
# tensorflow-transform                     0.14.0
# termcolor                                2.3.0
# terminado                                0.17.1
# textwrap3                                0.9.2
# threadpoolctl                            3.2.0
# tifffile                                 2023.9.18
# tinycss2                                 1.2.1
# toml                                     0.10.2
# tomli                                    2.0.1
# toolz                                    0.12.0
# tornado                                  6.3.3
# tqdm                                     4.66.1
# traitlets                                5.10.0
# typeguard                                2.13.3
# typer                                    0.9.0
# typing_extensions                        4.5.0
# typing-utils                             0.1.0
# tzdata                                   2023.3
# uri-template                             1.3.0
# uritemplate                              3.0.1
# urllib3                                  1.26.16
# uvicorn                                  0.23.2
# virtualenv                               20.21.0
# visions                                  0.7.5
# watchfiles                               0.20.0
# wcwidth                                  0.2.6
# webcolors                                1.13
# webencodings                             0.5.1
# websocket-client                         1.6.3
# Werkzeug                                 2.1.2
# wheel                                    0.41.2
# widgetsnbextension                       4.0.9
# witwidget                                1.8.1
# wordcloud                                1.9.2
# wrapt                                    1.15.0
# yarl                                     1.9.2
# ydata-profiling                          4.5.1
# zipp                                     3.17.0
# zstandard                                0.21.0
